In [2]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import cv2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#base_dir = '/content/drive/MyDrive/Training_1'
train_dir = '/content/drive/MyDrive/3classes_spinach/train'
validation_dir ='/content/drive/MyDrive/3classes_spinach/validation'

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=10, # rotation
                                   width_shift_range=0.2, # horizontal shift
                                   height_shift_range=0.2, # vertical shift
                                   zoom_range=0.2, # zoom
                                   horizontal_flip=True, # horizontal flip
                                   brightness_range=[0.2,1.2]) # brightness

valid_datagen = ImageDataGenerator( rescale = 1./255 )

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 8, class_mode = 'categorical', target_size = (150, 150))
validation_generator = valid_datagen.flow_from_directory(validation_dir, batch_size = 8, class_mode = 'categorical', target_size = (150, 150))

Found 320 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [6]:
from tensorflow.keras.applications.densenet import DenseNet201
base_model = DenseNet201(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers[:10]:
    layer.trainable=False
for layer in base_model.layers[10:]:
    layer.trainable=True


74836368/74836368 [==============================] - 3s 0us/step


In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras import regularizers

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.L2(0.03))(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(512, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.L2(0.03))(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(3, activation='softmax', kernel_initializer='he_uniform', kernel_regularizer=regularizers.L2(0.03))(x)

model = tf.keras.models.Model(base_model.input, x)

In [8]:
csv_path = r"./densenet/seq_model.csv"
log_dir = r"./densenet/seq_model"
save_weights_path = r"./densenet/Xception-trash-model-weight-ep-{epoch:02d}-val_loss-{val_loss:.4f}-val_acc-{val_accuracy:.4f}.h5"

In [9]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger
from tensorflow.keras.optimizers import Adam
from keras import backend as K

from keras.models import Model
from keras.regularizers import l2
import keras
import numpy as np
#checkpoint = ModelCheckpoint(save_weights_path, monitor='val_accuracy', verbose=1,
                             #save_weights_only=True, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.05, patience=10, verbose=1, min_lr=1e-4)
reducel = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=1e-4)

earlystop = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1)
early = EarlyStopping(monitor='val_loss',min_delta=1e-4, patience=10, verbose=1)

#logging = TensorBoard(log_dir=log_dir)
#csvlogger = CSVLogger(csv_path, append=True)

callbacks = [reduce_lr, earlystop]

In [10]:
def precision(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    '''Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def accuracy(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    return K.mean(K.equal(y_true, K.round(y_pred)))


In [11]:
#opt=tf.keras.optimizers.SGD(lr=0.25e-4, momentum = 0.7)
opt=tf.keras.optimizers.Adam(learning_rate=0.25e-3)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=opt)


In [15]:
inc_history = model.fit_generator(train_generator, validation_data = validation_generator,verbose=1, epochs =40)


<ipython-input-15-2f01a591d4da>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  inc_history = model.fit_generator(train_generator, validation_data = validation_generator,verbose=1, epochs =40)


Epoch 1/40


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <la

In [ ]:
import matplotlib.pyplot as plt

plt.plot(inc_history.history['accuracy'])
plt.plot(inc_history.history['val_accuracy'])
plt.title('DenseNet-201 Accuracy plot')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'], loc='lower right')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.plot(inc_history.history['loss'])
plt.plot(inc_history.history['val_loss'])
plt.title('DenseNet-201 Loss plot')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
model.save('Aishu_densenet.h5')

In [ ]:
reconstructed_model = keras.models.load_model("Aishu_densenet.h5")
reconstructed_model.layers[0].input_shape

In [ ]:
#Testing
import numpy as np
import tensorflow as tf
import seaborn as sn
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
input_shape = (150, 150, 3)
num_classes = 3
testset_dir = r"/content/drive/MyDrive/3classes_spinach/test_image"

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    testset_dir,
    target_size=(150,150),
    batch_size=400, # The number of test images
    class_mode='categorical')

In [ ]:
x_test, y_test = test_generator.__getitem__(0)


In [ ]:
len(x_test)

In [ ]:
test_true = np.argmax(y_test, axis=1)
test_pred = np.argmax(model.predict(x_test), axis=1)
print("DenseNet-201 Model Accuracy on test set: {:.4f}".format(accuracy(test_true, test_pred)))
print(classification_report(test_true, test_pred))

In [ ]:
y_pred = model.predict(x_test)
acc = np.count_nonzero(np.equal(np.argmax(y_pred, axis=1), np.argmax(y_test, axis=1)))/x_test.shape[0]
print(acc)

In [ ]:
pip install matplotlib==3.2.2

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(confusion_matrix(test_true, test_pred),index=['Healthy', 'Nitrogen', 'Potassium'], columns= ['Healthy', 'Nitrogen', 'Potassium'])
plt.figure(figsize = (8,8))
sn.set(font_scale=1.5)

sn.heatmap(df_cm, annot=True,annot_kws={"size": 19},fmt='2g')

In [ ]:
import tensorflow as tf
#from keras.preprocessing import image
from tensorflow.keras.utils import load_img
img_path = r'/content/drive/MyDrive/3classes_spinach/test_image/potassium/__081_IMG20220518204221.jpg'
#img = image.load_img(img_path, target_size=(150,150))
img = tf.keras.utils.load_img(img_path, target_size=(150,150))
#img = image.img_to_array(img, dtype=np.uint8)
img = tf.keras.utils.img_to_array(img, dtype=np.uint8)
img=np.array(img)/255.0

plt.title("Input Image")
plt.imshow(img)
plt.axis('off')
plt.show()

plt.axis('off')
plt.imshow(img.squeeze())
p=model.predict(img[np.newaxis, ...])
classes=['Healthy','Nitrogen','Potassium']
#print("Predicted shape",p.shape)
print("Maximum Probability: ",np.max(p[0], axis=-1))
print (p)
predicted_class = classes[np.argmax(p[0], axis=-1)]
print("Classified:",predicted_class)
plt.text(0, 0, predicted_class, fontsize = 22,color = 'r',bbox = dict(facecolor = 'yellow', alpha = 0.5))
plt.imshow(img)
plt.show()

In [ ]:
import tensorflow as tf
print(tf.__version__)